In [ ]:
# home task 1
# p | q => дешифрування без секретного ключа?
# answer:
# message == balancedmod(encrypted % p, p)

In [28]:
# example:

from sage.all import *
import sys
import math

# global variables known to everyone
N = 13      # best to be kept <= 20 (with bigger values my laptop tries to fly into outer space)
p = 3       # must be prime and relatively small <= 7
q = 3^6      # must be a large power of p
d = 7      # number of nonzero integer coefficients used in generated polynomials

# a class Zx of polynomials with integer coefficients and x as an unknown variable
Zx.<x> = ZZ[]

# ----------------------------------------------- AUXILIARY OPERATIONS ----------------------------------------------- 
def invertmodprime(f,p):
    ''' calculates an inversion of a polynomial modulo x^N-1 and then modulo p
        with assumption that p is prime.
        returns a Zx polynomial h such as convolution of h ~ f = 1 (mod p)                
        raises an exception if such Zx polynomial h doesn't exist'''

    T = Zx.change_ring(Integers(p)).quotient(x^N-1) # T is a quotient ring constructed from Zx after it's base being changed to Zp
    return Zx(lift(1 / T(f)))                       # with an use of an ideal x^N-1. Lift function converts Zp/x^N-1 back into Zp. 

def invertmodpowerofp(f,q):
    ''' calculates an inversion of a polynomial modulo x^N-1 and then modulo q
        with assumption that q is a power of 2.
        returns a Zx polynomial h such as convolution of h ~ f = 1 (mod q)                
        raises an exception if such Zx polynomial h doesn't exist'''

    assert q.is_power_of(p)     # asserting that q is a power of 2
    h = invertmodprime(f,p)     # first get an inversion of a polynomial like above with a prime number 2
    while True:
        r = balancedmod(convolution(h,f),q)         # get convolution of f and h (mod q) with balance
        if r == 1: return h                         # h * f = 1 (mod q), means h is a reciprocal of f       
        h = balancedmod(convolution(h,2 - r),q)     # get convolution of h and 2-r (mod q) with balance    

def balancedmod(f,q):
    ''' reduces every coefficient of a Zx polynomial f modulo q
        with additional balancing, so the result coefficients are integers in interval [-q/2, +q/2]
        more specifically: for an odd q [-(q-1)/2, +(q-1)/2], for an even q [-q/2, +q/2-1]. 
        returns Zx reduced polynomial'''

    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    return Zx(g)

def convolution(f,g):
    ''' performs a multiplication operation specific for NTRU, which works like a traditional polynomial multiplication
        with additional reduction of the result by x^N-1 (x^n is replaced by 1, x^n-1 by x, x^n-2 by x^2, ...)
        returns Zx polynomial'''
    
    return (f * g) % (x^N-1)

# ----------------------------------------------- BASIC SETUP -----------------------------------------------

def generate_polynomial(d):
    ''' generates a random polynomial with d nonzero coefficients
        returns Zx polynomial '''

    assert d <= N       # asserting that there are less nonzero coefficients given than number of all coefficients
    result = N*[0]      # vector variable to keep the result
    for j in range(d):  
        while True:
            r = randrange(N)            # get a random index < N    
            if not result[r]: break     # if there is no coefficient in this place of a vector 
        result[r] = 1-2*randrange(2)    # add a random number from a set {-1,0,1} in this place 
    return Zx(result)

# ----------------------------------------------- MAIN SETUP -----------------------------------------------
def generate_keys():
    ''' generates a public and private key pair, based on provided parameters
        returns Zx public key and a secret key as a tuple of Zx f (private key) and Zx F_p'''

        #   some polynomials are not invertible and as f and g are calculated randomly,
        #   it may be necessary to skip some invalid examples
    while True:
        try:
           # generate 2 random polynomials f and g with number of nonzero coefficients < given number
            f = generate_polynomial(d)
            g = generate_polynomial(d)

                # formula: find f_q, where: f_q (*) f = 1 (mod q)
                # assuming q is a power of 2                 
            f_q = invertmodpowerofp(f,q)

                # formula: find f_p, where: f_p (*) f = 1 (mod p) 
                # assuming p is a prime number 
            f_p = invertmodprime(f,p)  
            break
        
        except:
            pass 
    
        #formula: public key = F_q ~ g (mod q)
    public_key = balancedmod(p * convolution(f_q,g),q)

        #secret key is a tuple containing a private key (f) and variable f_p needed for decryption
    secret_key = f,f_p

    return public_key,secret_key

#---------------------------------- ENCRYPTION -----------------------------------------
def generate_message():
    ''' creates a polynomial from a random list of coefficients selected from a set {-1,0,1}  
        returns Zx polynomial'''
        
    #randrange(3) - 1 gives results from a set of {-1,0,1}, which is necessary for a proper decryption
    result = list(randrange(3) - 1 for j in range(N))
    return Zx(result)

def encrypt(message, public_key):
    ''' performs encryption of a given message using a provided public key
        returns Zx encrypted message'''

    # generate random polynomial with number of nonzero coefficients < N for adding extra noise    
    r = generate_polynomial(d)

    # formula: encrypted_message = p * r ~ public_key + message (mod q)
    # while performing modulo operation, balance coefficients of encrypted_message 
    # for the integers in interval [-q/2, +q/2]
    return balancedmod(convolution(public_key,r) + message,q)


def decrypt(encrypted_message, secret_key):
    ''' performs decryption of a given ciphertext using an own private key
        
        returns Zx decrypted message'''
    
    # private key - f; additional variable stored for decryption - f_p     
    f,f_p = secret_key
    
    # formula: a = f ~ encrypted_message (mod q)
    # balance coefficients of a for the integers in interval [-q/2, +q/2]
    a = balancedmod(convolution(encrypted_message,f),q)
     
    # formula: F_p ~ a (mod p) with additional balancing as above
    return balancedmod(convolution(a,f_p),p)

#----------------------------------------------------------------------------------------
#-------------------------------------- MAIN --------------------------------------------
#----------------------------------------------------------------------------------------

public_key, secret_key = generate_keys()
print(public_key, *secret_key, sep="\n")
message = generate_message()
print("MESSAGE: " + str(message))

encrypted_message = encrypt(message, public_key)
# print("ENCRYPTION: " + str(encrypted_message))
print("balanced ENCRYPTION: " + str(balancedmod(encrypted_message % p, p)))

print(message == balancedmod(encrypted_message % p, p), ":   message == balancedmod(encrypted_message % p, p)")

decrypted_message = decrypt(encrypted_message, secret_key)
print("DECRYPTION: " + str(decrypted_message))

if message == decrypted_message:
    print("SUCCESS")
else:
    print("FAIL")

189*x^12 + 285*x^11 + 45*x^10 - 201*x^9 - 162*x^8 + 348*x^7 + 111*x^6 + 195*x^5 - 168*x^4 - 93*x^3 - 147*x^2 + 177*x + 153
-x^12 - x^11 + x^10 - x^6 + x^5 - x^2 + x
2*x^12 + 2*x^11 + 2*x^9 + x^8 + 2*x^4 + 2
MESSAGE: -x^12 - x^11 - x^9 + x^8 - x^7 - x^3 + x^2 - x - 1
balanced ENCRYPTION: -x^12 - x^11 - x^9 + x^8 - x^7 - x^3 + x^2 - x - 1
True :   message == balancedmod(encrypted_message % p, p)
DECRYPTION: -x^12 - x^11 - x^9 + x^8 - x^7 - x^3 + x^2 - x - 1
SUCCESS


In [44]:
# home task 5
# multiplication (= convolution) - SageTeX (implementation, example)

In [20]:
# example:

N = 7

f = generate_polynomial(5)
fc = f.coefficients(sparse=False)
g = generate_polynomial(4)
gc = g.coefficients(sparse=False)

print(f*g % (x^N-1))

# lecture algorithm realization
ans_lst = [0] * N
for i in range(N):
    for j in range(N):
        ans_lst[(i+j)%N] += fc[i]*gc[j]
        
print(Zx(ans_lst))

# 3*x^6 - x^5 - 2*x^3 + x^2 + 2*x - 1
# 3*x^6 - x^5 - 2*x^3 + x^2 + 2*x - 1


3*x^6 - x^5 - 2*x^3 + x^2 + 2*x - 1
3*x^6 - x^5 - 2*x^3 + x^2 + 2*x - 1


In [ ]:
# home task 3
# example

In [45]:
from sage.all import *
import sys
import math

# global variables known to everyone
N = 13      # best to be kept <= 20 (with bigger values my laptop tries to fly into outer space)
p = 3       # must be prime and relatively small <= 7
q = 64      # must be a large power of p
d = 5      # number of nonzero integer coefficients used in generated polynomials

# a class Zx of polynomials with integer coefficients and x as an unknown variable
Zx.<x> = ZZ[]

# ----------------------------------------------- AUXILIARY OPERATIONS ----------------------------------------------- 
def invertmodprime(f,p):
    ''' calculates an inversion of a polynomial modulo x^N-1 and then modulo p
        with assumption that p is prime.
        returns a Zx polynomial h such as convolution of h ~ f = 1 (mod p)                
        raises an exception if such Zx polynomial h doesn't exist'''

    T = Zx.change_ring(Integers(p)).quotient(x^N-1) # T is a quotient ring constructed from Zx after it's base being changed to Zp
    return Zx(lift(1 / T(f)))                       # with an use of an ideal x^N-1. Lift function converts Zp/x^N-1 back into Zp. 

def invertmodpowerofp(f,q):
    ''' calculates an inversion of a polynomial modulo x^N-1 and then modulo q
        with assumption that q is a power of 2.
        returns a Zx polynomial h such as convolution of h ~ f = 1 (mod q)                
        raises an exception if such Zx polynomial h doesn't exist'''

    assert q.is_power_of(2)     # asserting that q is a power of 2
    h = invertmodprime(f,2)     # first get an inversion of a polynomial like above with a prime number 2
    while True:
        r = balancedmod(convolution(h,f),q)         # get convolution of f and h (mod q) with balance
        if r == 1: return h                         # h * f = 1 (mod q), means h is a reciprocal of f       
        h = balancedmod(convolution(h,2 - r),q)     # get convolution of h and 2-r (mod q) with balance    

def balancedmod(f,q):
    ''' reduces every coefficient of a Zx polynomial f modulo q
        with additional balancing, so the result coefficients are integers in interval [-q/2, +q/2]
        more specifically: for an odd q [-(q-1)/2, +(q-1)/2], for an even q [-q/2, +q/2-1]. 
        returns Zx reduced polynomial'''

    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    return Zx(g)

def convolution(f,g):
    ''' performs a multiplication operation specific for NTRU, which works like a traditional polynomial multiplication
        with additional reduction of the result by x^N-1 (x^n is replaced by 1, x^n-1 by x, x^n-2 by x^2, ...)
        returns Zx polynomial'''
    
    return (f * g) % (x^N-1)

# ----------------------------------------------- BASIC SETUP -----------------------------------------------

def generate_polynomial(d):
    ''' generates a random polynomial with d nonzero coefficients
        returns Zx polynomial '''

    assert d <= N       # asserting that there are less nonzero coefficients given than number of all coefficients
    result = N*[0]      # vector variable to keep the result
    for j in range(d):  
        while True:
            r = randrange(N)            # get a random index < N    
            if not result[r]: break     # if there is no coefficient in this place of a vector 
        result[r] = 1-2*randrange(2)    # add a random number from a set {-1,0,1} in this place 
    return Zx(result)

# ----------------------------------------------- MAIN SETUP -----------------------------------------------
def generate_keys():
    ''' generates a public and private key pair, based on provided parameters
        returns Zx public key and a secret key as a tuple of Zx f (private key) and Zx F_p'''

        #   some polynomials are not invertible and as f and g are calculated randomly,
        #   it may be necessary to skip some invalid examples
    while True:
        try:
           # generate 2 random polynomials f and g with number of nonzero coefficients < given number
            f = generate_polynomial(d)
            g = generate_polynomial(d)

                # formula: find f_q, where: f_q (*) f = 1 (mod q)
                # assuming q is a power of 2                 
            f_q = invertmodpowerofp(f,q)

                # formula: find f_p, where: f_p (*) f = 1 (mod p) 
                # assuming p is a prime number 
            f_p = invertmodprime(f,p)  
            break
        
        except:
            pass 
    
        #formula: public key = F_q ~ g (mod q)
    public_key = balancedmod(p * convolution(f_q,g),q)

        #secret key is a tuple containing a private key (f) and variable f_p needed for decryption
    secret_key = f,f_p

    return public_key,secret_key

#---------------------------------- ENCRYPTION -----------------------------------------
def generate_message():
    ''' creates a polynomial from a random list of coefficients selected from a set {-1,0,1}  
        returns Zx polynomial'''
        
    #randrange(3) - 1 gives results from a set of {-1,0,1}, which is necessary for a proper decryption
    result = list(randrange(3) - 1 for j in range(N))
    return Zx(result)

def encrypt(message, public_key):
    ''' performs encryption of a given message using a provided public key
        returns Zx encrypted message'''

    # generate random polynomial with number of nonzero coefficients < N for adding extra noise    
    r = -1+x^2+x^3+x^4-x^5-x^7
    # r = 1

    # formula: encrypted_message = p * r ~ public_key + message (mod q)
    # while performing modulo operation, balance coefficients of encrypted_message 
    # for the integers in interval [-q/2, +q/2]
    return balancedmod(convolution(public_key,r) + message,q)


def decrypt(encrypted_message, secret_key):
    ''' performs decryption of a given ciphertext using an own private key
        
        returns Zx decrypted message'''
    
    # private key - f; additional variable stored for decryption - f_p     
    f,f_p = secret_key
    
    # formula: a = f ~ encrypted_message (mod q)
    # balance coefficients of a for the integers in interval [-q/2, +q/2]
    a = balancedmod(convolution(encrypted_message,f),q)
     
    # formula: F_p ~ a (mod p) with additional balancing as above
    return balancedmod(convolution(a,f_p),p)

#----------------------------------------------------------------------------------------
#-------------------------------------- MAIN --------------------------------------------
#----------------------------------------------------------------------------------------

public_key, secret_key = generate_keys()
print(public_key, *secret_key, sep="\n")
print()

message1 = generate_message()
print("MESSAGE1: " + str(message1))
message2 = generate_message()
print("MESSAGE2: " + str(message2))
lst1 = message1.coefficients(sparse=False)
lst2 = message2.coefficients(sparse=False)
# print(len(lst1)==len(lst2))
lst_dif = [lst1[i] - lst2[i] for i in range(len(lst1))]
# print(lst1)
# print(lst2)
print("message1 - message2: ", lst_dif)
print()

encrypted_message1 = encrypt(message1, public_key)
print("ENCRYPTION: " + str(encrypted_message1))
encrypted_message2 = encrypt(message2, public_key)
print("ENCRYPTION: " + str(encrypted_message2))
lst11 = encrypted_message1.coefficients(sparse=False)
lst22 = encrypted_message2.coefficients(sparse=False)
# print(lst11)
# print(lst22)
lst_diff = [lst11[i] - lst22[i] for i in range(len(lst11))]
print("encrypted1 - encrypted2: ", lst_diff)
print()

decrypted_message1 = decrypt(encrypted_message1, secret_key)
# print("DECRYPTION1: " + str(decrypted_message1))
decrypted_message2 = decrypt(encrypted_message2, secret_key)
# print("DECRYPTION2: " + str(decrypted_message2))
# print()

if message1 == decrypted_message1:
    print("SUCCESS1")
else:
    print("FAIL1")
    
if message2 == decrypted_message2:
    print("SUCCESS2")
else:
    print("FAIL2")
    
# same bits to same bits

# MESSAGE1: -x^12 + x^10 + x^8 + x^7 + x^6 + x^3 + x^2 + x + 1
# MESSAGE2: x^12 + x^10 - x^8 + x^7 - x^6 + x^5 + x^4 - x + 1
# message1 - message2:  [0, 2, 1, 1, -1, -1, 2, 0, 2, 0, 0, 0, -2]

# ENCRYPTION: 11*x^12 - 9*x^11 - 5*x^10 - 11*x^8 + 4*x^7 + x^6 - 3*x^5 + 15*x^4 - 5*x^3 + 7*x^2 + 7*x - 5
# ENCRYPTION: 13*x^12 - 9*x^11 - 5*x^10 - 13*x^8 + 4*x^7 - x^6 - 2*x^5 + 16*x^4 - 6*x^3 + 6*x^2 + 5*x - 5
# encrypted1 - encrypted2:  [0, 2, 1, 1, -1, -1, 2, 0, 2, 0, 0, 0, -2]

-17*x^12 - 8*x^11 - 14*x^10 - 14*x^9 - 11*x^8 - 20*x^7 - 11*x^6 - 14*x^5 - 17*x^4 - 8*x^3 - 14*x^2 - 11*x - 8
x^12 + x^11 + x^8 + x^5 + x^4
x^10 + x^8 + x^7 + 2*x^5 + x^3 + x^2 + 1

MESSAGE1: -x^12 + x^10 + x^8 + x^7 + x^6 + x^3 + x^2 + x + 1
MESSAGE2: x^12 + x^10 - x^8 + x^7 - x^6 + x^5 + x^4 - x + 1
message1 - message2:  [0, 2, 1, 1, -1, -1, 2, 0, 2, 0, 0, 0, -2]

ENCRYPTION: 11*x^12 - 9*x^11 - 5*x^10 - 11*x^8 + 4*x^7 + x^6 - 3*x^5 + 15*x^4 - 5*x^3 + 7*x^2 + 7*x - 5
ENCRYPTION: 13*x^12 - 9*x^11 - 5*x^10 - 13*x^8 + 4*x^7 - x^6 - 2*x^5 + 16*x^4 - 6*x^3 + 6*x^2 + 5*x - 5
encrypted1 - encrypted2:  [0, 2, 1, 1, -1, -1, 2, 0, 2, 0, 0, 0, -2]

SUCCESS1
SUCCESS2
